In [8]:
import numpy as np
import gym
from gym.spaces import Discrete, Box
import torch.nn as nn
import torch
import torch.optim as opt
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import matplotlib
import torchvision.transforms as T
from torch.distributions import Categorical, Normal
from scipy import signal
import scipy
from torch import optim
import core

In [9]:
def statistics_scalar(x, with_min_and_max=False):
    """
    Get mean/std and optional min/max of scalar x across MPI processes.

    Args:
        x: An array containing samples of the scalar to produce statistics
            for.

        with_min_and_max (bool): If true, return min and max of x in 
            addition to mean and std.
    """
    x = np.array(x, dtype=np.float32)
    global_sum, global_n = np.sum(x), len(x)
    mean = global_sum / global_n

    global_sum_sq = np.sum((x - mean)**2)
    std = np.sqrt(global_sum_sq / global_n)  # compute global std

    if with_min_and_max:
        global_min =np.min(x) if len(x) > 0 else np.inf
        global_max =np.max(x) if len(x) > 0 else -np.inf
        return mean, std, global_min, global_max
    return mean, std

In [10]:
class VPGBuffer:
    def __init__(self, obs_dim, c, h, w, act_dim, size, gamma=0.99, lam=0.95):
        self.obs_buf = np.zeros(core.combined_shape(size, obs_dim), dtype=np.float32)
        self.frame_buf = np.zeros(core.combined_shape(size, c, h, w), dtype=np.float32)
        self.obs_buf = np.zeros(core.combined_shape(size, obs_dim), dtype=np.float32)
        self.act_buf = np.zeros(core.combined_shape(size, act_dim), dtype=np.float32)
        self.adv_buf = np.zeros(size, dtype=np.float32)
        self.rew_buf = np.zeros(size, dtype=np.float32)
        self.ret_buf = np.zeros(size, dtype=np.float32)
        self.val_buf = np.zeros(size, dtype=np.float32)
        self.logp_buf = np.zeros(size, dtype=np.float32)
        self.gamma, self.lam = gamma, lam
        self.ptr, self.path_start_idx, self.max_size = 0, 0, size

    def store(self, frame, obs, act, rew, val, logp):
        assert self.ptr < self.max_size     # buffer has to have room so you can store
        self.obs_buf[self.ptr] = obs
        self.frame_buf[self.ptr] = frame
        self.act_buf[self.ptr] = act
        self.rew_buf[self.ptr] = rew
        self.val_buf[self.ptr] = val
        self.logp_buf[self.ptr] = logp
        self.ptr += 1

    def finish_path(self, last_val=0):
        path_slice = slice(self.path_start_idx, self.ptr)
        rews = np.append(self.rew_buf[path_slice], last_val)
        vals = np.append(self.val_buf[path_slice], last_val)
        
        # the next two lines implement GAE-Lambda advantage calculation
        deltas = rews[:-1] + self.gamma * vals[1:] - vals[:-1]
        self.adv_buf[path_slice] = core.discount_cumsum(deltas, self.gamma * self.lam)
        
        # the next line computes rewards-to-go, to be targets for the value function
        self.ret_buf[path_slice] = core.discount_cumsum(rews, self.gamma)[:-1]
        
        self.path_start_idx = self.ptr

    def get(self):
        assert self.ptr == self.max_size    # buffer has to be full before you can get
        self.ptr, self.path_start_idx = 0, 0
        # the next two lines implement the advantage normalization trick
        adv_mean, adv_std = statistics_scalar(self.adv_buf)
        self.adv_buf = (self.adv_buf - adv_mean) / adv_std
        data = dict(frames=self.frame_buf, obs=self.obs_buf, act=self.act_buf, ret=self.ret_buf,
                    adv=self.adv_buf, logp=self.logp_buf)
        return {k: torch.as_tensor(v, dtype=torch.float32) for k,v in data.items()}


In [11]:
from torchvision.transforms import InterpolationMode
resize = T.Compose([T.ToPILImage(),
                    T.Resize(40, interpolation=InterpolationMode.BICUBIC),
                    T.ToTensor()])

In [16]:
def vpg(env, actor_critic=core.ActorCritic, ac_kwargs=dict(),  seed=0, 
        steps_per_epoch=4000, epochs=50, gamma=0.99, pi_lr=3e-4,
        vf_lr=1e-3, train_v_iters=80, lam=0.97, max_ep_len=1000,
        logger_kwargs=dict(), save_freq=10):
    
    # Random seed
    torch.manual_seed(seed)
    np.random.seed(seed)
    
    # Instantiate environment
    # env = env_fn()
    obs_dim = env.observation_space.shape
    act_dim = env.action_space.shape
    env.render()
    
    def get_screen():                             
        # Returned screen requested by gym is 400x600x3, but is sometimes larger
        # such as 800x1200x3. Transpose it into torch order (CHW).
        # time.sleep(2)
        # env = env()
        screen = env.render(mode='rgb_array')
        print(screen)
        screen  = screen.transpose((2, 0, 1))
        # screen = env.render(mode='rgb_array').transpose((2, 0, 1))
        # Cart is in the lower half, so strip off the top and bottom of the screen
        _, screen_height, screen_width = screen.shape
        screen = screen[:, int(screen_height*0.4):int(screen_height * 0.8)]
        view_width = int(screen_width * 0.6)
        def get_cart_location(screen_width):
            world_width = env.x_threshold * 2
            scale = screen_width / world_width
            return int(env.state[0] * scale + screen_width / 2.0)  # MIDDLE OF CART
        cart_location = get_cart_location(screen_width)
        if cart_location < view_width // 2:
            slice_range = slice(view_width)
        elif cart_location > (screen_width - view_width // 2):
            slice_range = slice(-view_width, None)
        else:
            slice_range = slice(cart_location - view_width // 2,
                                cart_location + view_width // 2)
        # Strip off the edges, so that we have a square image centered on a cart
        screen = screen[:, :, slice_range]
        # Convert to float, rescale, convert to torch tensor
        # (this doesn't require a copy)
        screen = np.ascontiguousarray(screen, dtype=np.float32) / 255
        screen = torch.from_numpy(screen)
        # Resize, and add a batch dimension (BCHW)
        resize = T.Compose([T.ToPILImage(),
                        T.Resize(40, interpolation=InterpolationMode.BICUBIC),
                        T.ToTensor()])
        return resize(screen).unsqueeze(0)

    _, c, h, w = get_screen().shape

    # Create actor-critic module
    ac = actor_critic(c, h, w, env.action_space, **ac_kwargs)

    # Set up experience buffer
    local_steps_per_epoch = int(steps_per_epoch)
    buf = VPGBuffer(obs_dim, act_dim, local_steps_per_epoch, gamma, lam)

    # Set up function for computing VPG policy loss
    def compute_loss_pi(data):
        frames, obs, act, adv, logp_old = data['frames'], data['obs'], data['act'], data['adv'], data['logp']

        # Policy loss
        pi, logp = ac.pi(c, h, w, act)
        loss_pi = -(logp * adv).mean()

        # Useful extra info
        approx_kl = (logp_old - logp).mean().item()
        ent = pi.entropy().mean().item()
        pi_info = dict(kl=approx_kl, ent=ent)
        
        return loss_pi, pi_info

    # Set up function for computing value loss
    def compute_loss_v(data):
        obs, ret = data['obs'], data['ret']
        return ((ac.v(obs) - ret)**2).mean()

    # Set up optimizers for policy and value function
    pi_optimizer = opt.Adam(ac.pi.parameters(), lr=pi_lr)
    vf_optimizer = opt.Adam(ac.v.parameters(), lr=vf_lr)

    def update():
        data = buf.get()

        # Get loss and info values before update
        pi_l_old, pi_info_old = compute_loss_pi(data)
        pi_l_old = pi_l_old.item()
        v_l_old = compute_loss_v(data).item()

        # Train policy with a single step of gradient descent
        pi_optimizer.zero_grad()
        loss_pi, pi_info = compute_loss_pi(data)
        loss_pi.backward()
        pi_optimizer.step()

        # Value function learning
        for i in range(train_v_iters):
            vf_optimizer.zero_grad()
            loss_v = compute_loss_v(data)
            loss_v.backward()
            vf_optimizer.step()

    # Prepare for interaction with environment
    frame, o, ep_ret, ep_len = get_screen(), env.reset(), 0, 0

    # Main loop: collect experience in env and update/log each epoch
    for epoch in range(epochs):
        for t in range(local_steps_per_epoch):
            a, v, logp = ac.step(torch.as_tensor(o, dtype=torch.float32))

            next_o, r, d, _ = env.step(a)
            frame_ = get_screen()
            ep_ret += r
            ep_len += 1

            # save and log
            buf.store(frame, o, a, r, v, logp)
            
            # Update obs (critical!)
            o = next_o
            frame = frame_

            timeout = ep_len == max_ep_len
            terminal = d or timeout
            epoch_ended = t==local_steps_per_epoch-1

            if terminal or epoch_ended:
                if epoch_ended and not(terminal):
                    print('Warning: trajectory cut off by epoch at %d steps.'%ep_len, flush=True)
                # if trajectory didn't reach terminal state, bootstrap value target
                if timeout or epoch_ended:
                    _, v, _ = ac.step(torch.as_tensor(o, dtype=torch.float32))
                else:
                    v = 0
                buf.finish_path(v)
                frame, o, ep_ret, ep_len = get_screen(), env.reset(), 0, 0


        # Save model
        if (epoch % save_freq == 0) or (epoch == epochs-1):
            torch.save(ac, f'./RL4_spinning_vpg')

        # Perform VPG update!
        update()

In [17]:
env = "CartPole-v0"
env = gym.make(env)

In [18]:
vpg(env, actor_critic=core.ActorCritic)

None


AttributeError: 'NoneType' object has no attribute 'transpose'

In [20]:
env
actor_critic=core.ActorCritic
ac_kwargs=dict()
seed=0 
steps_per_epoch=4000
epochs=50
gamma=0.99
pi_lr=3e-4
vf_lr=1e-3
train_v_iters=80
lam=0.97
max_ep_len=1000
logger_kwargs=dict()
save_freq=10
# Random seed
torch.manual_seed(seed)
np.random.seed(seed)

# Instantiate environment
# env = env_fn()
obs_dim = env.observation_space.shape
act_dim = env.action_space.shape
env.render()

def get_screen():                             
    # Returned screen requested by gym is 400x600x3, but is sometimes larger
    # such as 800x1200x3. Transpose it into torch order (CHW).
    # time.sleep(2)
    # env = env()
    screen = env.render(mode='rgb_array')
    print(screen)
    print(env.render('rgb_array'))
    screen  = screen.transpose((2, 0, 1))
    # screen = env.render(mode='rgb_array').transpose((2, 0, 1))
    # Cart is in the lower half, so strip off the top and bottom of the screen
    _, screen_height, screen_width = screen.shape
    screen = screen[:, int(screen_height*0.4):int(screen_height * 0.8)]
    view_width = int(screen_width * 0.6)
    def get_cart_location(screen_width):
        world_width = env.x_threshold * 2
        scale = screen_width / world_width
        return int(env.state[0] * scale + screen_width / 2.0)  # MIDDLE OF CART
    cart_location = get_cart_location(screen_width)
    if cart_location < view_width // 2:
        slice_range = slice(view_width)
    elif cart_location > (screen_width - view_width // 2):
        slice_range = slice(-view_width, None)
    else:
        slice_range = slice(cart_location - view_width // 2,
                            cart_location + view_width // 2)
    # Strip off the edges, so that we have a square image centered on a cart
    screen = screen[:, :, slice_range]
    # Convert to float, rescale, convert to torch tensor
    # (this doesn't require a copy)
    screen = np.ascontiguousarray(screen, dtype=np.float32) / 255
    screen = torch.from_numpy(screen)
    # Resize, and add a batch dimension (BCHW)
    resize = T.Compose([T.ToPILImage(),
                    T.Resize(40, interpolation=InterpolationMode.BICUBIC),
                    T.ToTensor()])
    return resize(screen).unsqueeze(0)

_, c, h, w = get_screen().shape

# Create actor-critic module
ac = actor_critic(c, h, w, env.action_space, **ac_kwargs)

# Set up experience buffer
local_steps_per_epoch = int(steps_per_epoch)
buf = VPGBuffer(obs_dim, act_dim, local_steps_per_epoch, gamma, lam)

# Set up function for computing VPG policy loss
def compute_loss_pi(data):
    frames, obs, act, adv, logp_old = data['frames'], data['obs'], data['act'], data['adv'], data['logp']

    # Policy loss
    pi, logp = ac.pi(c, h, w, act)
    loss_pi = -(logp * adv).mean()

    # Useful extra info
    approx_kl = (logp_old - logp).mean().item()
    ent = pi.entropy().mean().item()
    pi_info = dict(kl=approx_kl, ent=ent)
    
    return loss_pi, pi_info

# Set up function for computing value loss
def compute_loss_v(data):
    obs, ret = data['obs'], data['ret']
    return ((ac.v(obs) - ret)**2).mean()

# Set up optimizers for policy and value function
pi_optimizer = opt.Adam(ac.pi.parameters(), lr=pi_lr)
vf_optimizer = opt.Adam(ac.v.parameters(), lr=vf_lr)

def update():
    data = buf.get()

    # Get loss and info values before update
    pi_l_old, pi_info_old = compute_loss_pi(data)
    pi_l_old = pi_l_old.item()
    v_l_old = compute_loss_v(data).item()

    # Train policy with a single step of gradient descent
    pi_optimizer.zero_grad()
    loss_pi, pi_info = compute_loss_pi(data)
    loss_pi.backward()
    pi_optimizer.step()

    # Value function learning
    for i in range(train_v_iters):
        vf_optimizer.zero_grad()
        loss_v = compute_loss_v(data)
        loss_v.backward()
        vf_optimizer.step()

# Prepare for interaction with environment
frame, o, ep_ret, ep_len = get_screen(), env.reset(), 0, 0

# Main loop: collect experience in env and update/log each epoch
for epoch in range(epochs):
    for t in range(local_steps_per_epoch):
        a, v, logp = ac.step(torch.as_tensor(o, dtype=torch.float32))

        next_o, r, d, _ = env.step(a)
        frame_ = get_screen()
        ep_ret += r
        ep_len += 1

        # save and log
        buf.store(frame, o, a, r, v, logp)
        
        # Update obs (critical!)
        o = next_o
        frame = frame_

        timeout = ep_len == max_ep_len
        terminal = d or timeout
        epoch_ended = t==local_steps_per_epoch-1

        if terminal or epoch_ended:
            if epoch_ended and not(terminal):
                print('Warning: trajectory cut off by epoch at %d steps.'%ep_len, flush=True)
            # if trajectory didn't reach terminal state, bootstrap value target
            if timeout or epoch_ended:
                _, v, _ = ac.step(torch.as_tensor(o, dtype=torch.float32))
            else:
                v = 0
            buf.finish_path(v)
            frame, o, ep_ret, ep_len = get_screen(), env.reset(), 0, 0


    # Save model
    if (epoch % save_freq == 0) or (epoch == epochs-1):
        torch.save(ac, f'./RL4_spinning_vpg')

    # Perform VPG update!
    update()

None
None


AttributeError: 'NoneType' object has no attribute 'transpose'